In [15]:
import logging
import os

import numpy as np
from ConfigSpace.hyperparameters import UniformFloatHyperparameter

# Import ConfigSpace and different types of parameters
from smac.configspace import ConfigurationSpace
from smac.optimizer.acquisition import EI

from smac.facade.smac_ac_facade import SMAC4AC
from smac.runhistory.runhistory import RunHistory
from smac.scenario.scenario import Scenario
from smac.stats.stats import Stats
from smac.utils.io.traj_logging import TrajLogger

logging.basicConfig(level=logging.INFO)

In [16]:
def rosenbrock_2d(x):
    """The 2 dimensional Rosenbrock function as a toy model
    The Rosenbrock function is well know in the optimization community and
    often serves as a toy problem. It can be defined for arbitrary
    dimensions. The minimium is always at x_i = 1 with a function value of
    zero. All input parameters are continuous. The search domain for
    all x's is the interval [-5, 10].
    """

    x1 = x["x0"]
    x2 = x["x1"]

    val = 100.0 * (x2 - x1**2.0) ** 2.0 + (1 - x1) ** 2.0
    return val

In [17]:
# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()
x0 = UniformFloatHyperparameter("x0", -5, 10, default_value=-3)
x1 = UniformFloatHyperparameter("x1", -5, 10, default_value=-4)
cs.add_hyperparameters([x0, x1])

[x0, Type: UniformFloat, Range: [-5.0, 10.0], Default: -3.0,
 x1, Type: UniformFloat, Range: [-5.0, 10.0], Default: -4.0]

In [18]:
# Initialize scenario, using runcount_limit as budget.
original_scenario_dict = {
            "run_obj": "quality",  # we optimize quality (alternatively runtime)
            "runcount-limit": 5,  # max. number of function evaluations
            "cs": cs,  # configuration space
            "deterministic": True,
        }
original_scenario = Scenario(original_scenario_dict)

smac = SMAC4AC(
    scenario=original_scenario,
    run_id=1,
    rng=np.random.RandomState(42),
    tae_runner=rosenbrock_2d,
)

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2022-05-25_09:27:17_090106
INFO:smac.facade.smac_ac_facade.SMAC4AC:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.default_configuration_design.DefaultConfiguration:Running initial design for 1 configurations


In [19]:
def_value = rosenbrock_2d(cs.get_default_configuration())
print(f'default value : {def_value}')

default value : 16916.0


In [20]:
smac = SMAC4AC(scenario=original_scenario, run_id=1)

INFO:smac.facade.smac_ac_facade.SMAC4AC:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.default_configuration_design.DefaultConfiguration:Running initial design for 1 configurations


In [21]:
smac.optimize()
print("\nBudget exhausted! Starting restoring optimization ...\n")

INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.stats.stats.Stats:---------------------STATISTICS---------------------
INFO:smac.stats.stats.Stats:Incumbent changed: -1
INFO:smac.stats.stats.Stats:Submitted target algorithm runs: 1 / 5.0
INFO:smac.stats.stats.Stats:Finished target algorithm runs: 1 / 5.0
INFO:smac.stats.stats.Stats:Configurations: 1
INFO:smac.stats.stats.Stats:Used wallclock time: 0.04 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 0.00 / inf sec
INFO:smac.stats.stats.Stats:----------------------------------------------------
INFO:smac.facade.smac_ac_facade.SMAC4AC:Final Incumbent: None


FirstRunCrashedException: First run crashed, abort. Please check your setup -- we assume that your default configuration does not crashes. (To deactivate this exception, use the SMAC scenario option 'abort_on_first_run_crash'). Additional run info: {'traceback': 'Traceback (most recent call last):\n  File "/opt/anaconda3/envs/olympus/lib/python3.7/site-packages/smac/tae/base.py", line 226, in run_wrapper\n    instance_specific=run_info.instance_specific,\n  File "/opt/anaconda3/envs/olympus/lib/python3.7/site-packages/smac/tae/execute_ta_run_old.py", line 76, in run\n    seed=seed,\n  File "/opt/anaconda3/envs/olympus/lib/python3.7/site-packages/smac/tae/execute_ta_run_old.py", line 164, in _call_ta\n    raise TypeError("self.ta needs to be of type list or tuple, but is %s" % type(self.ta))\nTypeError: self.ta needs to be of type list or tuple, but is <class \'NoneType\'>\n', 'error': 'TypeError("self.ta needs to be of type list or tuple, but is <class \'NoneType\'>")'}

In [22]:
"""
Synthetic Function with few Hyperparameters
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
An example of applying SMAC to optimize a synthetic function (2d rosenbrock function).
We use the SMAC4BB facade because it is designed for black-box function optimization.
SMAC4BB uses a :term:`Gaussian Process<GP>` or a set of Gaussian Processes whose
hyperparameters are integrated by Markov-Chain Monte-Carlo as its surrogate model.
SMAC4BB works best on numerical hyperparameter configuration space and should not
be applied to the problems with large evaluation budgets (up to 1000 evaluations).
"""

import logging

logging.basicConfig(level=logging.INFO)

import numpy as np
from ConfigSpace.hyperparameters import UniformFloatHyperparameter

# Import ConfigSpace and different types of parameters
from smac.configspace import ConfigurationSpace
from smac.facade.smac_bb_facade import SMAC4BB
from smac.optimizer.acquisition import EI

# Import SMAC-utilities
from smac.scenario.scenario import Scenario


def rosenbrock_2d(x):
    """The 2 dimensional Rosenbrock function as a toy model
    The Rosenbrock function is well know in the optimization community and
    often serves as a toy problem. It can be defined for arbitrary
    dimensions. The minimium is always at x_i = 1 with a function value of
    zero. All input parameters are continuous. The search domain for
    all x's is the interval [-5, 10].
    """

    x1 = x["x0"]
    x2 = x["x1"]

    val = 100.0 * (x2 - x1**2.0) ** 2.0 + (1 - x1) ** 2.0
    return val


# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()
x0 = UniformFloatHyperparameter("x0", -5, 10, default_value=-3)
x1 = UniformFloatHyperparameter("x1", -5, 10, default_value=-4)
cs.add_hyperparameters([x0, x1])

# Scenario object
scenario = Scenario(
    {
        "run_obj": "quality",  # we optimize quality (alternatively runtime)
        "runcount-limit": 10,  # max. number of function evaluations
        "cs": cs,  # configuration space
        "deterministic": True,
    }
)

# Use 'gp' or 'gp_mcmc' here
model_type = "gp"

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = rosenbrock_2d(cs.get_default_configuration())
print("Default Value: %.2f" % def_value)

# Optimize, using a SMAC-object
print("Optimizing! Depending on your machine, this might take a few minutes.")
smac = SMAC4BB(
    scenario=scenario,
    model_type=model_type,
    rng=np.random.RandomState(42),
    acquisition_function=EI,  # or others like PI, LCB as acquisition functions
    tae_runner=rosenbrock_2d,
)

smac.optimize()

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2022-05-25_09:28:01_471264
INFO:smac.facade.smac_bb_facade.SMAC4BB:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.sobol_design.SobolDesign:Running initial design for 2 configurations
INFO:smac.facade.smac_bb_facade.SMAC4BB:<class 'smac.facade.smac_bb_facade.SMAC4BB'>
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 5311.5619


Default Value: 16916.00
Optimizing! Depending on your machine, this might take a few minutes.


INFO:smac.intensification.intensification.Intensifier:Challenger (4744.681) is better than incumbent (5311.5619) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  x0 : -0.21980144549161196 -> 0.666692873028599
INFO:smac.intensification.intensification.Intensifier:  x1 : 7.335338136181235 -> 7.332563252406933
INFO:smac.intensification.intensification.Intensifier:Challenger (939.3255) is better than incumbent (4744.681) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  x0 : 0.666692873028599 -> 2.263660637291685
INFO:smac.intensification.intensification.Intensifier:  x1 : 7.332563252406933 -> 8.186395068270798
INFO:smac.intensification.intensification.Intensifier:Challenger (920.0862) is better than incumbent (939.3255) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:

Configuration(values={
  'x0': 2.2708123182821804,
  'x1': 8.187217627538942,
})